## Preproccess and tranning

1. preproccess https://www.kaggle.com/drzhuzhe/hubmap-tf-with-tpu-efficientunet-256-tfrecord
2. my trainning notebook https://www.kaggle.com/drzhuzhe/hubmap-efficientnet-and-linknet-train
3. my predict visualization https://www.kaggle.com/drzhuzhe/ananlysis-predict


## Reffernce notebook

* 1. Tensorflow HuBMAP - Hacking the Kidney competition starter kit:
* https://www.kaggle.com/wrrosa/hubmap-tf-with-tpu-efficientunet-512x512-tfrecs (how to create training and inference tfrecords) https://www.kaggle.com/wrrosa/hubmap-tf-with-tpu-efficientunet-512x512-train (training pipeline)
* 2. best public LB notebook https://www.kaggle.com/vgarshin/kidney-unet-model-keras-inference
* https://github.com/vgarshin/kaggle_kidney/blob/master/kidney_train.ipynb
* This infer note book is as same as wrrosa's work too

* 3. TODO https://www.kaggle.com/matjes/hubmap-efficient-sampling-deepflash2-train

## Approach 

1. https://www.kaggle.com/c/hubmap-kidney-segmentation/discussion/200955
2. https://www.kaggle.com/c/hubmap-kidney-segmentation/discussion/200626




# Versions
* V1 - V2 use effientnetb4 + unet baseline form wrrosa’s idea (LB .907)
* V3 try  efficientb0 + linknet from vgarshin‘s idea (LB .915)
* v4 ~intend to try deepflash's approach~ but just try efficientnetb0 + unet
* v5 - v22 fix preprocess bug use 256 pics
* v22-v40 tuning params  (LB .920)
* v40-v43 add TTA for tfrecord (LB .921)
* v44 add extenal data 

# Parameters
Read parameteres from notebook output, actually only **DIM** is used:

In [ ]:
# 改成用我自己的
#mod_path = '/kaggle/input/hubmap-efficientnet-and-linknet-train/'
mod_path = '/kaggle/input/hubmap-efficientnet-and-linknet-train/'
import yaml
import pprint
with open(mod_path+'params.yaml') as file:
    P = yaml.load(file, Loader=yaml.FullLoader)
    pprint.pprint(P)
    
THRESHOLD = 0.3 # preds > THRESHOLD
WINDOW = 1024

MIN_OVERLAP =  1 / 16
NEW_SIZE = P['DIM']
REDUCE = int(WINDOW / NEW_SIZE)
EXPAND = 4

SUBMISSION_MODE = 'PUBLIC_TFREC' # PUBLIC_TFREC or FULL
# 'PUBLIC_TFREC' = use created tfrecords for public test set with MIN_OVERLAP = 300 tiling 1024-512, ignore other (private test) data
# 'FULL' do not use tfrecords, just full submission 

CHECKSUM = True # compute mask sum for each image
selected_models = [0, 1, 2, 3, 4]
#selected_models = [0]
TTAS = [0, 1, 2, 3]
#TTAS = [0]
VOTERS = 0.5
def flip(img, axis=0):
    if axis == 1:
        return img[::-1, :, ]
    elif axis == 2:
        return img[:, ::-1, ]
    elif axis == 3:
        return img[::-1, ::-1, ]
    else:
        return img

In [ ]:
!ls ../input

# Metrics

In [ ]:
import json

with open(mod_path + 'metrics.json') as json_file:
    M = json.load(json_file)
print('Model run datetime: '+M['datetime'])
print('OOF val_dice_coe: ' + str(M['oof_dice_coe']))

# Packages

In [ ]:
! pip install ../input/kerasapplications/keras-team-keras-applications-3b180cb -f ./ --no-index -q
! pip install ../input/efficientnet/efficientnet-1.1.0/ -f ./ --no-index -q
import numpy as np
import pandas as pd
import os
import glob
import gc

import rasterio
from rasterio.windows import Window

import pathlib
from tqdm.notebook import tqdm
import cv2

import tensorflow as tf
import efficientnet as efn
import efficientnet.tfkeras
#import tensorflow.keras.backend as K

In [ ]:
#glob.glob(mod_path+'*.h5')

In [ ]:
#a = np.array([[[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[7,8,9]]])
"""
a = np.array(
    [[[[1],[2],[3]],
     [[4],[5],[6]],
     [[7],[8],[9]]],
    [[[1],[2],[3]],
     [[4],[5],[6]],
     [[7],[8],[9]]]])
"""
#a[0][::-1, :, ]
#print(flip(a[1], axis=2))
#print(K.reverse(a[0],axes=2))
#r = []
#for x in a:
#    x[...] = flip(x, axis=2)
#print(np.array(r))
#print(a)
#a = flip(a, axis=3)
#print(a)


# Functions

In [ ]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)

# Models

In [ ]:
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
fold_models = []
# for fold_model_path in glob.glob(mod_path+'.h5'): 
for selected_models_idx in selected_models:
    fold_model_path = f'/kaggle/input/hubmap-efficientnet-and-linknet-train/model-fold-{selected_models_idx}.h5'
    print(fold_model_path)
    fold_models.append(tf.keras.models.load_model(fold_model_path,compile = False))
print(len(fold_models), fold_models)

# Tfrecords functions

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
image_feature = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'x1': tf.io.FixedLenFeature([], tf.int64),
    'y1': tf.io.FixedLenFeature([], tf.int64)
}
def _parse_image(example_proto, tta_mode=0):
    example = tf.io.parse_single_example(example_proto, image_feature)
    image = tf.reshape( tf.io.decode_raw(example['image'],out_type=np.dtype('uint8')), (P['DIM'],P['DIM'], 3))
    
    # 左右
    if tta_mode == 2:
        image = tf.image.flip_left_right(image)
    # 上下
    if tta_mode == 1:
        image = tf.image.flip_up_down(image)
    # 反转
    if tta_mode == 3:
        image = tf.image.rot90(image, k=2)
        
    return image, example['x1'], example['y1']

def load_dataset(filenames, ordered=True, tta_mode=0):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda ex: _parse_image(ex, tta_mode=tta_mode))
    return dataset

def get_dataset(FILENAME, tta_mode=0):
    dataset = load_dataset(FILENAME, tta_mode=tta_mode)
    dataset  = dataset.batch(64)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Results

In [ ]:
p = pathlib.Path('../input/hubmap-kidney-segmentation')
subm = {}

for i, filename in tqdm(enumerate(p.glob('test/*.tiff')), 
                        total = len(list(p.glob('test/*.tiff')))):
    
    print(f'{i+1} Predicting {filename.stem}')
    
    dataset = rasterio.open(filename.as_posix(), transform = identity)
    
    # predict on 4 * 4 * 256 image to save infer time
    tile_size = int(NEW_SIZE * EXPAND)
    tile_resized = int(tile_size * REDUCE)
             
    preds = np.zeros(dataset.shape, dtype=np.uint8)    
    
    if SUBMISSION_MODE == 'PUBLIC_TFREC' and MIN_OVERLAP == 300 and WINDOW == 1024 and NEW_SIZE == 256:
        print('SUBMISSION_MODE: PUBLIC_TFREC')
        fnames = glob.glob('/kaggle/input/hubmap-tf-with-tpu-efficientunet-256-tfrecord/test/'+filename.stem+'*.tfrec')
        
        if len(fnames)>0: # PUBLIC TEST SET
            for FILENAME in fnames:
                pred = None
                
                for tta_mode in TTAS:
                    for fold_model in fold_models:
                        tmp = fold_model.predict(get_dataset(FILENAME, tta_mode=tta_mode))/len(fold_models)  
                        # 还原
                        for x in tmp:
                            x[...] = flip(x, axis=tta_mode)                    
                        if pred is None:
                            pred = tmp
                        else:
                            pred += tmp
                        del tmp
                        gc.collect()
                    
                pred = pred / len(TTAS)
                
                pred = tf.cast((tf.image.resize(pred, (WINDOW,WINDOW)) > THRESHOLD),tf.bool).numpy().squeeze()

                idx = 0
                for img, X1, Y1 in get_dataset(FILENAME):
                    for fi in range(X1.shape[0]):
                        x1 = X1[fi].numpy()
                        y1 = Y1[fi].numpy()
                        preds[x1:(x1+WINDOW),y1:(y1+WINDOW)] += pred[idx]
                        idx += 1
                        
        else: # IGNORE PRIVATE TEST SET (CREATE TFRECORDS IN FUTURE)
            pass
    else:
        print('SUBMISSION_MODE: FULL')
        slices = make_grid(dataset.shape, window=tile_resized, min_overlap=int( tile_resized * MIN_OVERLAP ))
                    
        if dataset.count != 3:
            print('Image file with subdatasets as channels')
            layers = [rasterio.open(subd) for subd in dataset.subdatasets]
            
        for (x1,x2,y1,y2) in slices:
            if dataset.count == 3:
                image = dataset.read([1,2,3],
                            window=Window.from_slices((x1,x2),(y1,y2)))
                image = np.moveaxis(image, 0, -1)
            else:
                image = np.zeros((tile_resized, tile_resized, 3), dtype=np.uint8)
                for fl in range(3):
                    image[:,:,fl] = layers[fl].read(window=Window.from_slices((x1,x2),(y1,y2)))
                    
            image = cv2.resize(image, (tile_size, tile_size),interpolation = cv2.INTER_AREA)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            #image = np.expand_dims(image, 0)

            pred = np.zeros((tile_size, tile_size), dtype=np.float32)            
            ### tta s 
            #"""                  
            for tta_mode in TTAS:
                img_aug = flip(image, axis=tta_mode)
                img_aug = np.expand_dims(img_aug, 0)
                """
                my model is training without pixel/255, 
                if you did, plase uncommit next line 
                or you will get all predicts very small value under THRESHOLD so get a empty submit
                """ 
                #img_aug = img_aug.astype(np.float32) / 255
                for fold_model in fold_models:
                    pred_aug = np.squeeze(fold_model.predict(img_aug))
                    pred += flip(pred_aug, axis=tta_mode)
            pred /= (len(fold_models) * len(TTAS))           
            # tta e

            pred = cv2.resize(pred, (tile_resized, tile_resized))
            preds[x1:x2,y1:y2] += (pred > THRESHOLD).astype(np.uint8)
                    
        del fold_model, image, pred, img_aug, pred_aug; gc.collect()

    preds = (preds > VOTERS).astype(np.uint8)
    
    subm[i] = {'id':filename.stem, 'predicted': rle_encode_less_memory(preds)}
    
    if CHECKSUM:
        print('Checksum: '+ str(np.sum(preds)))
    
    del preds
    gc.collect();

# Making submission

In [ ]:
# kaggle 的单人资源似乎是抢占式的

submission = pd.DataFrame.from_dict(subm, orient='index')
submission.to_csv('submission.csv', index=False)
submission.head()